In [76]:
%%time
import mpmath
from sympy import *
from math import factorial
from IPython.display import display,Math

CPU times: total: 0 ns
Wall time: 0 ns


In [77]:
sigma1,sigma2,sigma3,sigma4,sigma5,sigma6,sigma7,sigma8 = symbols('Sigma_1 Sigma_2 Sigma_3 Sigma_4 Sigma_5 Sigma_6 Sigma_7 Sigma_8', commutative=False)
sigma1T,sigma2T,sigma3T,sigma4T,sigma5T,sigma6T,sigma7T,sigma8T = symbols('Sigma_1^\dagger Sigma_2^\dagger Sigma_3^\dagger Sigma_4^\dagger Sigma_5^\dagger Sigma_6^\dagger Sigma_7^\dagger Sigma_8^\dagger',commutative=False)

lam = symbols('lambda')


sigmaOps = [1,sigma1,sigma2,sigma3,sigma4,sigma5,sigma6,sigma7,sigma8]
sigmaOpsT = [1,sigma1T,sigma2T,sigma3T,sigma4T,sigma5T,sigma6,sigma7,sigma8]

In [78]:
def printTerms(tlist):
    print()
    for index,term in enumerate(tlist):
        print(f'Lambda {index+2}: ')
        display(Math(latex(term)))
        print()

In [79]:
lamList = []

def getExpr(n):   
    terms_dict = {}
    
    tLeft = 0
    for s in range(1,n-1):
        tLeft +=  lam**s * sigmaOpsT[s]
    noTRight = 0
    for s in range(1,n-1):
        noTRight +=  lam**s * sigmaOps[s]

    innerProd = tLeft * noTRight

    innerSum = 0
    for m in range(1,n-1):
        innerSum += lam**m * (sigmaOpsT[m] + sigmaOps[m])
    innerSum += innerProd

    rightSum = 1
    for l in range(1,n-1):
        rightSum -= lam**l * sigmaOpsT[l]
  
    expr = 0
    for k in range(n-1):
        expr += innerSum**k

    outerSum = 0 #not part of pinv
    for s in range(1,n):
        outerSum +=  lam**s * sigmaOps[s]

    expr *= rightSum

    expr *= outerSum

    expr = expand(expr)
    print('Expansion done')
    # Extract terms
    orderedTerms = expr.as_ordered_terms()
    for cycle,term in enumerate(orderedTerms):
        # Collect terms by the leading var `lam`
        coeff, var = term.as_coeff_mul()
        # Check if `lam` is part of the term
        # Use as_powers_dict to get powers of all symbols in the term
        powers_dict = term.as_powers_dict()
        
        # Check if 'lam' is in the powers_dict and get its power
        lam_power = powers_dict.get(lam, 0)
        if lam_power < n:
            # Store the term in the dictionary with index `lam_power - 1`
            if lam_power - 1 in terms_dict:
                terms_dict[lam_power - 1] += term
            else:
                terms_dict[lam_power - 1] = term
    print('dict finished')
    # Create a list with terms at appropriate indexes
    terms_list = [lam * terms_dict.get(i, 0) for i in range(max(terms_dict.keys()) + 1)]
    return terms_list
        

# Regular TCL Expansion

In [80]:
lam2List = []

def getExprNormal(n):  
    terms_dict = {}
    innerSum = 0
    for s in range(1,n):
        innerSum +=  lam**s * sigmaOps[s]
    expr = 0
    for k in range(n):
        expr += innerSum**k
    expr = expand(expr)
    print('Expansion done')
    # Extract terms
    orderedTerms = expr.as_ordered_terms()
    for cycle,term in enumerate(orderedTerms):
        # Collect terms by the leading factor `lam`
        coeff, var = term.as_coeff_mul()
        # Check if `lam` is part of the term
        # Use as_powers_dict to get powers of all symbols in the term
        powers_dict = term.as_powers_dict()
        
        # Check if 'lam' is in the powers_dict and get its power
        lam_power = powers_dict.get(lam, 0)
        if lam_power < n:
            # Store the term in the dictionary with index `lam_power - 1`
            if lam_power - 1 in terms_dict:
                terms_dict[lam_power - 1] += term
            else:
                terms_dict[lam_power - 1] = term
    print('dict finished')
    # Create a list with terms at appropriate indexes
    terms_list = [factor(lam * terms_dict.get(i, 0)) for i in range(max(terms_dict.keys()) + 1)]
    return terms_list

# Solving for each term

In [81]:
import itertools
from collections import OrderedDict
from copy import deepcopy

In [82]:
%%time

def sumCombosHelper(n,comboSet,currList,max):
    if len(currList) > max:
        return
    if sum(currList) == n:
        sortedList = sorted(currList)
        if sortedList not in comboSet:
            comboSet.append(sortedList)
        return
    else:
        for i in range(1,n+1):
            currList.append(i)   
            sumCombosHelper(n,comboSet,currList,max)
            currList.pop()
            
def sumCombos(n,max):
    combos = []
    sumCombosHelper(n,combos,[],max)
    return combos

def getIndices(n,s):
    iList = [i for i in range(2*(n-2))]

    listT = [i for i in range(n)]

    for combo in itertools.product(listT, repeat=2):
        if combo[0] + combo[1] <= n-2 and 0 not in combo:
            iList.append(combo)
    
    return tuple(itertools.product(iList,repeat=s))


CPU times: total: 0 ns
Wall time: 0 ns


In [83]:
def getTCLPlusTerm(n):
    #array of indices
    indices = [getIndices(n,i) for i in range(1,n)]
    #powers
    powerSet = []

    if n == 2:
        return lam**2*sigma1
    
    for p in range(n-1):
        sumComboList = sumCombos(p,p)
        for sum in sumComboList:
            #get all index combos of length sum
            ind = indices[len(sum)-1]
            if p == 0:
                ind = indices[0]
            for idxCombo in ind:
               
                powerDict = []
                lamTotal=0
                #populate the powerDict at all possible indices, do terms inside power
                for index,num in enumerate(sum):
                    idxTuple = [idxCombo[index],num]
                    powerDict.append(idxTuple)
                    if isinstance(idxCombo[index],int):
                        lamTotal += num * (idxCombo[index] % (n-2) + 1)
                    elif isinstance(idxCombo[index],tuple):
                        lamTotal += num * (idxCombo[index][0]+idxCombo[index][1])
                    
                 
                #terms outside the power
                for i in range(0,n):
                    for j in range(1,n):
                        #valid combo
                        if lamTotal + i + j == n-1:
                            pDict = powerDict.copy()
                            prev=()
                            #removing combos of squares e.g. (0,1),(0,1) vs. (0,2)
                            for id in range (len(pDict)-1,0,-1):
                                ent = pDict[id][0]
                                prev = pDict[id-1][0]
                                if ent == prev:
                                    pDict[id-1][1]+=pDict[id][1]
                                    del pDict[id]

                            innerTuple = ('inner',i)
                            outerTuple = ('outer',j)
                            pDict.append(innerTuple)
                            pDict.append(outerTuple)
  
                            appender = {p: pDict}
                            
                            if appender not in powerSet:
                                powerSet.append(appender)

                        elif lamTotal + i + j > n-1:
                            break

    
    pSum = 0       
    for termSet in powerSet:       
        for power,dictionary in termSet.items():
            
            pTerm = 1
            #coeff = factorial(power)
            for key,value in dictionary:
                if isinstance(key,int):
                    if key < n-2:
                        pTerm *= lam**((key%(n-2)+1)*value) * sigmaOps[key+1]**value
                    elif n-2 <= key:
                        pTerm *= lam**((key % (n-2)+1)*value) * sigmaOpsT[(key % (n-2)+1)]**value
                elif isinstance(key,tuple):
                    pTerm*=(lam**(key[0]+key[1])*value) * sigmaOpsT[key[0]]**value * sigmaOps[key[1]]**value
                    #coeff /= factorial(value)
                    
                else:
                    if key == 'inner':
                        pTerm*=lam**value*sigmaOpsT[value]
                        if value != 0:
                            pTerm *= -1
                    if key == 'outer':
                        pTerm*=lam**value*sigmaOps[value]
            pSum += expand(pTerm) #*coeff
                
    ordered_terms = pSum.as_ordered_terms()
    terms_dict = {}
    for cycle,term in enumerate(ordered_terms):

        # Collect terms by the leading factor `lam`
        coeff, var = term.as_coeff_mul()
        # Check if `lam` is part of the term
        # Use as_powers_dict to get powers of all symbols in the term
        powers_dict = term.as_powers_dict()
        
        # Check if 'lam' is in the powers_dict and get its power
        lam_power = powers_dict.get(lam, 0)
        if lam_power < n:
            # Store the term in the dictionary with index `lam_power - 1`
            if lam_power - 1 in terms_dict:
                terms_dict[lam_power - 1] += term
            else:
                terms_dict[lam_power - 1] = term
    print('dict finished')
    # Create a list with terms at appropriate indexes
    term = factor(lam * terms_dict.get(n-2, 0)) 
    return term


def createPlusList(n):
    termList = []
    for i in range(2,n+1):
        termList.append(getTCLPlusTerm(i))
    return termList
                
                            

            
             

In [84]:
def getDiff(n):
    dList = []
    reg = getExprNormal(n)
    plus = createPlusList(n)
    for i in range(n-1):
        dList.append(simplify(plus[i] - reg[i]))
    return dList

In [85]:
%%time
printTerms(createPlusList(6))

dict finished
dict finished
dict finished
dict finished

Lambda 2: 


<IPython.core.display.Math object>


Lambda 3: 


<IPython.core.display.Math object>


Lambda 4: 


...


Lambda 5: 


<IPython.core.display.Math object>


Lambda 6: 


<IPython.core.display.Math object>


CPU times: total: 1 s
Wall time: 1.82 s


# Getting T Terms

In [ ]:
%%time
printTerms(getDiff(8))

Expansion done
dict finished
dict finished
dict finished
dict finished
dict finished
dict finished
